In [ ]:
# https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289
import panel as pn
import altair as alt
from altair import datum
import pandas as pd
from vega_datasets import data
import datetime as dt
import plotly.express as px


alt.renderers.enable('default')
# pn.extension('vega')
pn.extension('plotly')

source = data.stocks()
source = pd.DataFrame(source)
print (source.head())

# create list of company names (tickers) to use as options
tickers = ['AAPL', 'GOOG', 'IBM', 'MSFT']
# this creates the dropdown widget
ticker = pn.widgets.Select(name='Company', options=tickers)

# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2001, 1, 1), end=dt.datetime(2010, 1, 1),
    value=(dt.datetime(2001, 1, 1), dt.datetime(2010, 1, 1))
)

title = '### Stock Price Dashboard'
subtitle = 'This dashboard allows you to select a company and date range to see stock prices.'

@pn.depends(ticker.param.value, date_range_slider.param.value)
def get_plot(ticker, date_range):
     # Load and format the data
     df = source # define df
     df['date'] = pd.to_datetime(df['date'])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_range_slider.value[0] 
     end_date = date_range_slider.value[1] 
     # create filter mask for the dataframe
     mask = (df['date'] > start_date) & (df['date'] <= end_date)
     df = df.loc[mask] # filter the dataframe
    
     # create the Altair chart object
     # chart = alt.Chart(df).mark_line().encode(x='date', y='price', tooltip=alt.Tooltip(['date','price'])).transform_filter((datum.symbol == ticker))
     
     fig = px.line(x=df[df.symbol == ticker].date, y=df[df.symbol == ticker].price)
     # fig.show()

     return fig

dashboard = pn.Row(pn.Column(title, subtitle, ticker, date_range_slider),
    get_plot # our draw chart function!
)

# dashboard.servable()
dashboard

In [ ]:
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2001, 1, 1), end=dt.datetime(2010, 1, 1),
    value=(dt.datetime(2001, 1, 1), dt.datetime(2010, 1, 1))
)

@pn.depends(date_range_slider.param.value)
def callback(date_range):
    chart = alt.Chart(source[source.symbol == 'MSFT']).mark_line().encode(
        x='date',
        y='price'
    )
    # just not rendering the chart
    # return chart
    return date_range

# row
pn.Row(date_range_slider, callback)


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def f(x, y=True):
    return x * y

interact(f, x=10)

In [ ]:
# https://stackoverflow.com/questions/39258580/ipywidget-with-date-slider
import ipywidgets as widgets
import pandas as pd
from datetime import datetime

start_date = datetime(2018, 4, 24)
end_date = datetime(2018, 5, 24)

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

selection_range_slider

In [ ]:
# https://plotly.com/python/widget-app/
# from plotly.widgets import GraphWidget
from chart_studio.widgets import GraphWidget

g = GraphWidget('https://plot.ly/~kevintest/1192/')

In [ ]:
import altair as alt

print (source.head())
df = source[source.symbol == 'MSFT']

import plotly.express as px
# fig = px.line(x=df.date, y=df.price)
fig = px.scatter(x=df.date, y=df.price)
fig.show()

chart = alt.Chart(df).mark_line().encode(
    x='date',
    y='price'
)

chart

In [ ]:
import plotly.graph_objects as go
import numpy as np
df = pd.read_csv("cars.csv")

f = go.FigureWidget([go.Scatter(y = df['City mpg'], x = df['City mpg'], mode = 'markers')])
scatter = f.data[0]
N = len(df)
scatter.x = scatter.x + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.y = scatter.y + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.marker.opacity = 0.5


def selection_fn(trace, points, selector):
    print (points, selector)
    t.data[0].cells.values = [df.loc[points.point_inds][col] for col in ['ID','Classification','Driveline','Hybrid']]

scatter.on_selection(selection_fn)

f.show()

In [ ]:
# https://community.plotly.com/t/selecting-points-in-python/5295/3
# notebook graph selection
import plotly.graph_objs as go
import plotly.offline as py

import pandas as pd
import numpy as np
from ipywidgets import interactive, HBox, VBox

py.init_notebook_mode()
# df = pd.read_csv('https://raw.githubusercontent.com/jonmmease/plotly_ipywidget_notebooks/master/notebooks/data/cars/cars.csv')
# df.to_csv("cars.csv", index=False)
df = pd.read_csv("cars.csv")

f = go.FigureWidget([go.Scatter(y = df['City mpg'], x = df['City mpg'], mode = 'markers')])
scatter = f.data[0]
N = len(df)
scatter.x = scatter.x + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.y = scatter.y + np.random.rand(N)/10 *(df['City mpg'].max() - df['City mpg'].min())
scatter.marker.opacity = 0.5

def update_axes(xaxis, yaxis):
    scatter = f.data[0]
    scatter.x = df[xaxis]
    scatter.y = df[yaxis]
    with f.batch_update():
        f.layout.xaxis.title = xaxis
        f.layout.yaxis.title = yaxis
        scatter.x = scatter.x + np.random.rand(N)/10 *(df[xaxis].max() - df[xaxis].min())
        scatter.y = scatter.y + np.random.rand(N)/10 *(df[yaxis].max() - df[yaxis].min())
    
axis_dropdowns = interactive(update_axes, yaxis = df.select_dtypes('int64').columns, xaxis = df.select_dtypes('int64').columns)

# Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(
    header=dict(values=['ID','Classification','Driveline','Hybrid'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in ['ID','Classification','Driveline','Hybrid']],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

def selection_fn(trace,points,selector):
    t.data[0].cells.values = [df.loc[points.point_inds][col] for col in ['ID','Classification','Driveline','Hybrid']]

scatter.on_selection(selection_fn)

# Put everything together
x = VBox((HBox(axis_dropdowns.children),f,t))
f.show()